In [1]:
import os

import torch
from scipy import ndimage
import math
import json
import numpy as np

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation, gum_jaw_separation, vertical_separation, bounding_teeth_on_origin
from utils.preprocess import recovery_rotated_bounding, xyxy2xywh, get_image_by_labels, get_labels_by_image
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
load_dotenv()



True

In [2]:
target_labels = ['caries', 'endo', 'post', 'crown']
image_labels_df = get_image_by_labels(target_labels)[target_labels]

image_labels_df


,caries,endo,post,crown
filename,,,,
00006145,True,False,False,False
00008026,True,True,True,True
00008113,True,True,True,True
00008117,True,False,False,False
00008131,True,True,False,False
...,...,...,...,...
202012050412159442018_0690727A,False,True,True,True
202012050412229432017_0781002A,False,True,False,False
202012051012069362000_0760503A,False,True,False,True


In [3]:
data_dir = Path('..') / '..' / 'Datasets' / 'phase-2'
tooth_position_dict = {
    0: 'left',
    1: 'middle',
    2: 'right'
}
model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')

for filename in tqdm(image_labels_df.index):
    filepath_image = data_dir / f'{filename}.jpg'
    filepath_json = data_dir / f'{filename}.json'

    results = model(filepath_image)

    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][filename]
    teeth_roi_split_teeth = teeth_roi['split_teeth']
    for target_roi in teeth_roi_images:
        target_roi_image = target_roi['image']
        flag = target_roi['flag']
        tooth_position = tooth_position_dict[target_roi['number']]
        im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
        im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

        isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False, save=True, filename=filename)


YOLOv5  torch 1.12.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)



requirements: D:\Users\douli\anaconda3\envs\dentist-CV-main\Lib\site-packages\yolov5\requirements.txt not found, check failed.


Fusing layers... 
Model summary: 270 layers, 7041205 parameters, 0 gradients
Adding AutoShape... 
100%|██████████| 365/365 [37:20<00:00,  6.14s/it]
